In [1]:
from Acesso import Login
from Query import Query
import pandas as pd
from Moeda import Moeda
import warnings

warnings.filterwarnings('ignore')

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Vendas':
    
    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    IF MONTH(@DTBASE)=1

        BEGIN

            SET @DTFIM=DATEADD(DAY,-365,DATEADD(DAY,DAY(@DTBASE)*-1,@DTBASE))

            SET @DTINICIO=DATEADD(DAY,-365,DATEADD(DAY,DAY(@DTFIM)*-1,@DTFIM))

        END;


    ELSE

        BEGIN

            SET @DTFIM=@DTBASE

            SET @DTINICIO=DATEADD(DAY,-365,DATEADD(DAY,DAY(@DTFIM)*-1,@DTFIM))

        END;

    SELECT * FROM netfeira.vw_targetestatistico
    WHERE [ID Situação]='FA' AND [Tipo de Operação]='VENDAS'
	AND [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM
    
    """,
    
    'Receber':
    
    """
    
    SELECT * FROM netfeira.vw_contareceber
    WHERE Situação<>'CANCELADO'
        
    """,
    
    'Calendario':
    
    """
    
    SELECT * FROM netfeira.vw_calend
    
    """,
    
    'Data':
    
    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    IF DAY(@DTBASE)=1

        BEGIN

            SET @DTFIM=DATEADD(DAY,DAY(@DTBASE)*-1,@DTBASE)

            SET @DTINICIO=DATEADD(DAY,1,DATEADD(DAY,DAY(@DTFIM)*-1,@DTFIM))

        END;


    ELSE

        BEGIN

            SET @DTFIM=DATEADD(DAY,DAY(@DTBASE)*-1,@DTBASE)

            SET @DTINICIO=DATEADD(DAY,1,DATEADD(DAY,DAY(@DTFIM)*-1,@DTFIM))

        END;

    SELECT @DTFIM AS [Anterior DTFIM],@DTINICIO AS [Anterior DTINICIO],
    @DTBASE DTFIM,
    DATEADD(DAY,1,DATEADD(DAY,DAY(@DTBASE)*-1,@DTBASE)) AS DTINICIO
    
    """,
    
    'Notas':
    
    """
    
    SELECT NFe 
	FROM netfeira.vw_targetestatistico
    WHERE [ID Situação]='FA' AND [Tipo de Operação]='VENDAS'
	GROUP BY NFe
    
    """
    
}

In [3]:
df=sql.CriarTabela(kwargs=querys)

In [4]:
for c in df.keys():
    
    display(df[c].columns)
    
    pass

Index(['ID Usuário', 'Data de Emissão', 'Data de Faturamento',
       'Data de Entrega', 'Origem', 'ID Empresa', 'ID Cliente', 'ID Vendedor',
       'Tipo de Pedido', 'Tipo de Operação', 'Tabelas', 'Pedido', 'NFe',
       'Condição de Pagamento', 'SKU', 'Seq', 'Qtde', 'Local de Estoque',
       'Unid. VDA', 'Fator VDA', 'Qtde. VDA', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Total Venda', 'Custo CUE', 'Margem Bruta R$',
       'Custo Capado CUE', 'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$',
       'Verba R$', 'Margem CTB R$', 'Total Geral AV', 'Peso Bruto KG',
       'Peso Líquido KG', 'ID Situação', 'Situação'],
      dtype='object')

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento',
       'Data de Fluxo', 'Título', 'Serie', 'Condição de Pagamento',
       'Tipo de Pagamento', 'ID Cliente', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Matriz', 'ID Segmento', 'Situação', 'Valor',
       'Desconto R$', 'Multa R$', 'Juros R$', 'Abatimento R$', 'Taxa R$',
       'Valor Líquido', 'Pago R$', 'Status do Título', 'ID Situação', 'Dias',
       'Alerta', 'Freq. Pagto'],
      dtype='object')

Index(['Data', 'Dia Útil', 'Data Trabalhada', 'Ano', 'ID Mês', 'Mês',
       'Mês Resumido', 'Dia', 'Mês Meta', 'ID Semana', 'Semana',
       'Trimestre Ano', 'Semestre Ano', 'Semana Ano'],
      dtype='object')

Index(['Anterior DTFIM', 'Anterior DTINICIO', 'DTFIM', 'DTINICIO'], dtype='object')

Index(['NFe'], dtype='object')

In [5]:
notas=df['Notas']['NFe'].unique().tolist()

In [6]:
df['Receber']=df['Receber'].loc[df['Receber']['Título'].isin(notas)]

In [7]:
df['Data']

,Anterior DTFIM,Anterior DTINICIO,DTFIM,DTINICIO
0,2023-04-30,2023-04-01,2023-05-26,2023-05-01


In [8]:
dt_antfim=df['Data']['Anterior DTFIM'].max()

dt_antinicio=df['Data']['Anterior DTINICIO'].max()

dt_fim=df['Data']['DTFIM'].max()

dt_inicio=df['Data']['DTINICIO'].max()

In [9]:
df['Analise']=df['Receber'].loc[~df['Receber']['Data de Emissão'].between(dt_inicio,dt_fim)]

df['Analise']=df['Analise'].loc[(df['Analise']['ID Situação'].isin(['AB','PL']))&(df['Analise']['Data de Fluxo'].between(dt_inicio,dt_fim))]

df['Analise']

,Data de Emissão,Data de Vencimento,Data de Pagamento,Data de Fluxo,Título,Serie,Condição de Pagamento,Tipo de Pagamento,ID Cliente,CNPJ,Razão Social,Nome Fantasia,Matriz,ID Segmento,Situação,Valor,Desconto R$,Multa R$,Juros R$,Abatimento R$,Taxa R$,Valor Líquido,Pago R$,Status do Título,ID Situação,Dias,Alerta,Freq. Pagto
348045,2023-04-10,2023-05-25,NaT,2023-05-25,1001178,,DEPOSITO BANCARIO 45 DIAS,None,8978,11317530000346,VIRTUS COMERCIO DE ALIMENTOS LTDA,ST MARCHE ALDEIA DA,ST MARCHE,0001,EM ABERTO,241.10,12.06,0.0,0.0,0.0,0.0,229.04,0.0,VENCIDO,AB,1,ATÉ 5 DIAS,INADIMPLÊNCIA
348046,2023-04-10,2023-05-25,NaT,2023-05-25,1001192,,BOLETO BANCARIO 45 DIAS,None,15466,03205493000356,MERCEARIA CHAMA LTDA,CHAMA SUPERMERCADOS,CHAMA,0001,EM ABERTO,914.22,45.71,0.0,0.0,0.0,0.0,868.51,0.0,VENCIDO,AB,1,ATÉ 5 DIAS,INADIMPLÊNCIA
348047,2023-04-10,2023-05-25,NaT,2023-05-25,1001220,,DEPOSITO BANCARIO 45 DIAS,None,28804,18303320001581,CRESCO COMERCIO DE ALIMENTOS LTDA,ST MARCHE VL OLIMPIA,ST MARCHE,0001,EM ABERTO,222.48,11.12,0.0,0.0,0.0,0.0,211.36,0.0,VENCIDO,AB,1,ATÉ 5 DIAS,INADIMPLÊNCIA
348335,2023-04-11,2023-05-26,NaT,2023-05-26,1001548,,DEPOSITO BANCARIO 45 DIAS,None,25829,09151220000433,FORTIS COMERCIO DE ALIMENTOS LTDA,ST MARCHE PINHEIROS,ST MARCHE,0001,EM ABERTO,627.42,31.37,0.0,0.0,0.0,0.0,596.05,0.0,A VENCER,AB,0,None,A VENCER
348336,2023-04-11,2023-05-26,NaT,2023-05-26,1001629,,DEPOSITO BANCARIO 45 DIAS,None,23787,11317530001075,VIRTUS COMERCIO DE ALIMENTOS LTDA,ST MARCHE PINHEIROS,ST MARCHE,0001,EM ABERTO,279.66,13.98,0.0,0.0,0.0,0.0,265.68,0.0,A VENCER,AB,0,None,A VENCER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443370,2023-04-25,2023-05-23,NaT,2023-05-23,1004552,,BOLETO BANCARIO 28 DIAS,None,23060,18208506000156,LANCHONETE PONTO DO ACAI JARDIM MIRNA LTDA.,PONTO DO ACAI,None,0010,EM ABERTO,2337.60,0.00,0.0,0.0,0.0,0.0,2337.60,0.0,VENCIDO,AB,3,ATÉ 5 DIAS,INADIMPLÊNCIA
443415,2023-04-27,2023-05-25,NaT,2023-05-25,1005117,,BOLETO BANCARIO 28 DIAS,None,25165,35940438000487,REINALDO CHICAROLLI MARTINS SUPERMERCADOS EIRELI,MERCADO CHIMAR,MERCADO CHIMAR,0001,EM ABERTO,453.71,0.00,0.0,0.0,0.0,0.0,453.71,0.0,VENCIDO,AB,1,ATÉ 5 DIAS,INADIMPLÊNCIA
443416,2023-04-27,2023-05-25,NaT,2023-05-25,1005232,,BOLETO BANCARIO 28 DIAS,None,29977,50274296000107,GAC DOCES E ALIMENTOS LTDA,GAC HAVANNA TATUAPE,HAVANNA,0009,EM ABERTO,453.06,0.00,0.0,0.0,0.0,0.0,453.06,0.0,VENCIDO,AB,1,ATÉ 5 DIAS,INADIMPLÊNCIA
443429,2023-04-28,2023-05-05,NaT,2023-05-05,1005430,,BOLETO BANCARIO 07 DIAS,None,20885,30845720000119,ESPACO CASA BLANCA LANC SECOS E MOL LTDA,ESPACO CASA BLANCA,None,0004,EM ABERTO,224.52,0.00,0.0,0.0,0.0,0.0,224.52,0.0,VENCIDO,AB,21,16 A 30 DIAS,INADIMPLÊNCIA


In [10]:
round(df['Analise']['Valor'].sum(),2)

208078.8

In [11]:
df['Mensal']=df['Receber'].loc[(df['Receber']['Data de Emissão'].between(dt_inicio,dt_fim))]

df['Mensal']=df['Mensal'].merge(df['Calendario'],left_on='Data de Fluxo',right_on='Data',how='inner')

df['Mensal']=df['Mensal'].groupby(['ID Mês','Mês'],as_index=False).agg({'Valor':'sum'})

df['Mensal']['Total']=df['Mensal']['Valor'].sum()

df['Mensal']['Perc']=df['Mensal'].apply(lambda info: round(info['Valor']/info['Total'],4)*100 if info['Total']>0 else 0,axis=1)

df['Mensal']

,ID Mês,Mês,Valor,Total,Perc
0,5,MAIO,2000120.44,4124746.28,48.49
1,6,JUNHO,1991336.77,4124746.28,48.28
2,7,JULHO,114247.75,4124746.28,2.77
3,8,AGOSTO,19041.32,4124746.28,0.46


In [12]:
df['Mensal']['Valor'].sum()

4124746.28

In [28]:
df['Liquidado']=df['Receber'].loc[(df['Receber']['ID Situação']=='LQ')&(df['Receber']['Data de Emissão'].between(dt_inicio,dt_fim))]

df['Liquidado']=df['Liquidado'].loc[(df['Liquidado']['Data de Fluxo'].between(dt_inicio,dt_fim))]

df['Liquidado']['Valor'].sum()

1232488.08

# Status do cliente

In [14]:
tabela='Status'

df[tabela]=df['Receber'].loc[df['Receber']['ID Situação'].isin(['LQ','AB','PL'])]

df['Temp']=df[tabela].groupby(['ID Cliente','Freq. Pagto'],as_index=False).agg({'Valor Líquido':'sum'})

df['Temp']['Total']=df['Temp'].apply(lambda info: df['Temp'].loc[df['Temp']['ID Cliente']==info['ID Cliente'],'Valor Líquido'].sum(),axis=1)

df['Temp']=df['Temp'].loc[df['Temp']['Freq. Pagto']!='A VENCER']

df['Temp']['Perc']=df['Temp'].apply(lambda info: round(info['Valor Líquido']/info['Total'],4)*100 if info['Total']>0 else 0,axis=1)

colunas=df['Temp']['Freq. Pagto'].unique().tolist()

df['Temp']=df['Temp'].pivot(index=['ID Cliente','Total'],columns='Freq. Pagto',values='Perc').reset_index()

codigos=[]

indice=51

temp_df=pd.DataFrame()

for c in colunas:
            
    df['T']=df['Temp'].loc[(df['Temp'][c]>=indice)&(~df['Temp']['ID Cliente'].isin(codigos))]
    
    df['T']['Status']=c
    
    temp_df=pd.concat([temp_df,df['T'][['ID Cliente','Total','Status']]],axis=0,ignore_index=True)
    
    codigos=df['Temp'].loc[(df['Temp'][c]>=indice),'ID Cliente'].unique().tolist()
    
    #break
    
    pass

codigos=temp_df['ID Cliente'].unique().tolist()

df[tabela]=temp_df

df['Temp']=df['Receber'].loc[(~df['Receber']['ID Cliente'].isin(codigos))&(df['Receber']['ID Situação']=='LQ')].groupby(['ID Cliente','Freq. Pagto'],as_index=False).agg({'Valor Líquido':'sum'})

df['Temp']['Total']=df['Temp'].apply(lambda info: df['Temp'].loc[df['Temp']['ID Cliente']==info['ID Cliente'],'Valor Líquido'].sum(),axis=1)

df['Temp']['Perc']=df['Temp'].apply(lambda info: round(info['Valor Líquido']/info['Total'],4)*100 if info['Total']>0 else 0,axis=1)

df['Temp']=df['Temp'].pivot(index=['ID Cliente','Total'],columns='Freq. Pagto',values='Perc').reset_index()

df['Temp']['Status']='DEVEDOR OCASIONAL'

df[tabela]=pd.concat([df[tabela],df['Temp'][['ID Cliente','Total','Status']]],axis=0,ignore_index=True)

df[tabela]


Freq. Pagto,ID Cliente,Total,Status
0,6,362.94,ANTECIPADO
1,1566,4883.52,ANTECIPADO
2,1658,193422.76,ANTECIPADO
3,1671,424342.97,ANTECIPADO
4,2350,416282.92,ANTECIPADO
...,...,...,...
12927,30047,1196.22,DEVEDOR OCASIONAL
12928,30051,2018.68,DEVEDOR OCASIONAL
12929,30057,483.24,DEVEDOR OCASIONAL
12930,30064,4605.39,DEVEDOR OCASIONAL


In [15]:
df['Status'].groupby(['Status'],as_index=False).agg({'ID Cliente':'count','Total':'sum'}).sort_values('ID Cliente',ascending=False,ignore_index=True)

Freq. Pagto,Status,ID Cliente,Total
0,NO PRAZO,5433,1.461200e+08
1,FORA DO PRAZO,4490,2.482768e+07
2,ANTECIPADO,1882,2.179953e+07
3,DEVEDOR OCASIONAL,989,2.138520e+07
4,INADIMPLÊNCIA,138,2.360301e+05


# Condição de Pagamento

In [16]:
df['Receber']['Tipo de Pagamento'].unique().tolist()

['DINHEIRO',
 'CARTÃO DE CRÉDITO',
 'COBRANÇA BANCÁRIA',
 'ABATIMENTO ENCONTRO CONTAS',
 None,
 'CHEQUE',
 'DEPOSITO EM CONTA',
 'CHEQUE TERCEIROS',
 'DESC/ABAT INSTRUÇÃO BANCÁRIA',
 'NOTA PROMISSÓRIA',
 'CARTÃO DE DÉBITO']

In [17]:
tabela='Pagamentos'

df[tabela]=df['Receber'].loc[df['Receber']['ID Situação'].isin(['LQ'])]

df['Temp']=df[tabela].groupby(['Condição de Pagamento','Freq. Pagto'],as_index=False).agg({'Valor Líquido':'sum'})

df['Temp']['Total']=df['Temp'].apply(lambda info: df['Temp'].loc[df['Temp']['Condição de Pagamento']==info['Condição de Pagamento'],'Valor Líquido'].sum(),axis=1)

df['Temp']=df['Temp'].loc[df['Temp']['Freq. Pagto']!='A VENCER']

df['Temp']['Perc']=df['Temp'].apply(lambda info: round(info['Valor Líquido']/info['Total'],4)*100 if info['Total']>0 else 0,axis=1)

colunas=df['Temp']['Freq. Pagto'].unique().tolist()

df['Temp']=df['Temp'].pivot(index=['Condição de Pagamento','Total'],columns='Freq. Pagto',values='Perc').reset_index()

codigos=[]

indice=51

temp_df=pd.DataFrame()

for c in colunas:
            
    df['T']=df['Temp'].loc[(df['Temp'][c]>=indice)&(~df['Temp']['Condição de Pagamento'].isin(codigos))]
    
    df['T']['Status']=c
    
    temp_df=pd.concat([temp_df,df['T'][['Condição de Pagamento','Total','Status']]],axis=0,ignore_index=True)
    
    codigos=df['Temp'].loc[(df['Temp'][c]>=indice),'Condição de Pagamento'].unique().tolist()
    
    #break
    
    pass

codigos=temp_df['Condição de Pagamento'].unique().tolist()

df[tabela]=temp_df

df[tabela]

Freq. Pagto,Condição de Pagamento,Total,Status
0,A VISTA QUINZENA,22143.96,ANTECIPADO
1,DEPOSITO BANCARIO,399143.36,ANTECIPADO
2,DEPOSITO BANCARIO 15 DIAS,55620.93,ANTECIPADO
3,VENDA FUNCIONARIO,24941.32,ANTECIPADO
4,A VISTA,7652661.52,FORA DO PRAZO
5,CORNERSHOP,24795.04,FORA DO PRAZO
6,CREDITO PDV,850676.69,FORA DO PRAZO
7,DEPOSITO BANCARIO 30 DIAS,3001440.05,FORA DO PRAZO
8,DEPOSITO BANCARIO 40 DIAS,598547.44,FORA DO PRAZO
9,DEPOSITO BANCARIO 45 DIAS,2040557.91,FORA DO PRAZO
